# Tuning of curve paramaters

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import VBox, Accordion, Button, Output
import pprint
import importlib
import config
import ast

importlib.reload(config)

MIN_COST = 0
MAX_COST = 1

# --- Generalized Cauchy membership function ---
def generalized_cauchy(x, a, b, c):
    y = 1.0 / (1.0 + np.abs((x - c) / (a + 1e-9))**(2*b))
    return MIN_COST + (MAX_COST - MIN_COST) * y


# --- Slider helper ---
def make_sliders(layer_name):
    params = config.TRANSFORM_PARAMS[layer_name]
    sliders = {}
    for k, (lo, hi) in config.PARAM_RANGES.items():
        sliders[k] = widgets.FloatSlider(
            value=params.get(k, (lo+hi)/2),
            min=lo, max=hi, step=(hi-lo)/200,
            description=k, readout_format=".3g", continuous_update=True
        )
    return sliders

# --- Draw curve ---
def draw_curve(ax, x, params):
    y = generalized_cauchy(x, params['a'], params['b'], params['c'])
    ax.plot(x, y)
    ax.set_ylim(0, MAX_COST*1.1)
    ax.grid(True)
    ax.set_xlabel("x")
    ax.set_ylabel("cost")

# --- Build UI ---
layers_ui = {}
x_range = np.linspace(0, 180, 500)

for layer_name in config.TRANSFORM_PARAMS.keys():
    sliders = make_sliders(layer_name)
    out = Output()
    
    def update_plot(change=None, layer=layer_name, out_box=out, sliders=sliders):
        params = {k: s.value for k,s in sliders.items()}
        with out_box:
            out_box.clear_output(wait=True)
            fig, ax = plt.subplots(1,1,figsize=(5,3))
            draw_curve(ax, x_range, params)
            ax.set_title(layer)
            plt.show()
    
    for s in sliders.values():
        s.observe(update_plot, names='value')
    
    update_plot()  # initial plot
    
    layers_ui[layer_name] = {"sliders": sliders, "out": out}
    vbox = VBox(list(sliders.values()) + [out])
    layers_ui[layer_name]["vbox"] = vbox

accordion = Accordion(children=[layers_ui[name]["vbox"] for name in layers_ui])
for i, name in enumerate(layers_ui):
    accordion.set_title(i, name)
display(accordion)

# --- Save button ---
def replace_assignment_in_file(pyfile, varname, new_py_value_str):
    with open(pyfile,"r") as f:
        src = f.read()
    tree = ast.parse(src)
    target_node = None
    for node in tree.body:
        if isinstance(node, ast.Assign):
            for t in node.targets:
                if isinstance(t, ast.Name) and t.id == varname:
                    target_node = node
    if target_node is None:
        new_src = src.rstrip() + f"\n\n{varname} = {new_py_value_str}\n"
    else:
        start = target_node.lineno-1
        end = getattr(target_node,"end_lineno", target_node.lineno)
        lines = src.splitlines(True)
        lines[start:end] = [f"{varname} = {new_py_value_str}\n"]
        new_src = "".join(lines)
    # backup
    with open(pyfile+".bak","w") as f:
        f.write(src)
    with open(pyfile,"w") as f:
        f.write(new_src)


def on_save(_):
    new_params = {layer:{k:s.value for k,s in layers_ui[layer]["sliders"].items()} for layer in layers_ui}
    replace_assignment_in_file("config.py", "TRANSFORM_PARAMS", pprint.pformat(new_params, indent=4))
    print("✅ Saved to config.py (backup at config.py.bak)")

save_btn = Button(description="Save to config.py", button_style="success")
save_btn.on_click(on_save)
display(save_btn)


Accordion(children=(VBox(children=(FloatSlider(value=3.0, description='a', max=90.0, readout_format='.3g', ste…

Button(button_style='success', description='Save to config.py', style=ButtonStyle())